# 🔁 Simple RNN Walkthrough for Sentiment Analysis
This notebook shows step-by-step how an RNN processes a sentence word by word using manual matrix calculations to update hidden states.

In [2]:
import torch
import torch.nn.functional as F

## 1. Define Vocabulary and Sentence

In [3]:
# Simple vocabulary and input sentence
vocab = {"i": 0, "love": 1, "this": 2, "movie": 3}
sentence = ["i", "love", "this", "movie"]
indices = torch.tensor([vocab[word] for word in sentence])

## 2. Define Embeddings

In [4]:
# Manually created word embeddings (embedding_dim = 4)
embedding_matrix = torch.tensor([
    [0.1, 0.2, 0.3, 0.4],   # i
    [0.5, 0.6, 0.7, 0.8],   # love
    [0.2, 0.1, 0.0, -0.1],  # this
    [0.0, 0.1, 0.2, 0.3],   # movie
])

## 3. Define a Simple Manual RNN Cell

In [14]:
# Simple RNN cell with fixed weights
class RNNCell:
    def __init__(self, input_dim, hidden_dim):
        self.Wx = torch.tensor([
            [0.1, 0.2, 0.3],
            [0.0, -0.1, 0.2],
            [0.2, 0.1, 0.0],
            [-0.1, 0.1, 0.1]
        ])  # shape: (input_dim, hidden_dim)

        self.Wh = torch.tensor([
            [0.1, 0.0, 0.1],
            [0.2, 0.1, -0.1],
            [-0.1, 0.2, 0.0]
        ])  # shape: (hidden_dim, hidden_dim)

        self.b = torch.zeros(3)

    def step(self, x_t, h_prev):
        return torch.tanh(x_t @ self.Wx + h_prev @ self.Wh + self.b)

## 4. Run the RNN Over the Sentence

In [15]:
# Instantiate and initialize hidden state
rnn = RNNCell(input_dim=4, hidden_dim=3)
h_t = torch.zeros(3)

print("Time Step | x_t (Embedding)            | h_t (Hidden State)")
print("-------------------------------------------------------------")
for idx in indices:
    x_t = embedding_matrix[idx]
    h_t = rnn.step(x_t, h_t)
    x_t_str = str(x_t.numpy())
    h_t_str = str(h_t.detach().numpy())
    print(f"{idx.item():>9} | {x_t_str:^25} | {h_t_str}")

Time Step | x_t (Embedding)            | h_t (Hidden State)
-------------------------------------------------------------
        0 |     [0.1 0.2 0.3 0.4]     | [0.029991   0.06988589 0.10955847]
        1 |     [0.5 0.6 0.7 0.8]     | [0.11550264 0.21546964 0.33283275]
        2 |   [ 0.2  0.1  0.  -0.1]   | [0.05131581 0.10769425 0.0599314 ]
        3 |     [0.  0.1 0.2 0.3]     | [0.03066767 0.06267345 0.04433308]


In [16]:
# 5. Simple Sentiment Analysis Output Layer

# Let's assume a simple linear layer to map the final hidden state to a sentiment score
# We'll use 1 neuron: positive if >0, negative if <0

# W_out is a parameter of the output layer, not produced by the loop above.
# The loop above computes h_t (the final hidden state), which is then used here.
# There are 3 weights in W_out because the hidden state h_t has dimension 3 (hidden_dim=3).
# Each element of h_t is multiplied by its corresponding weight in W_out to compute the sentiment score.
W_out = torch.tensor([0.5, -0.3, 0.8])  # shape: (hidden_dim,)
b_out = torch.tensor(0.0)

# Compute sentiment score from the last hidden state
sentiment_score = (h_t * W_out).sum() + b_out

# Interpret the result
sentiment = "positive" if sentiment_score.item() > 0 else "negative"

print(f"\nSentiment score: {sentiment_score.item():.4f}")
print(f"Predicted sentiment: {sentiment}")


Sentiment score: 0.0320
Predicted sentiment: positive
